In [2]:
import modin.pandas as pd
import numpy as np
import pickle
import re
import random
import hanja
import localSummarizer

from sklearn.feature_extraction.text import TfidfVectorizer
from konlpy.tag import Kkma
from konlpy.utils import pprint

In [3]:
def df1pre(text):
    # 불필요한 문자 제거 및 대체
    punct = {'…':'...',
            '“':'"', '”':'"',
            "‘":"'", "’":"'",
            "‘":"'", "’":"'",
            '\t':'', '\n':'', '\r':' ',
            '/':'',
            '▲':'', '■':'', '●':'', '○':'',
            '(사진)':'', '동영상 뉴스':''}
    for p in punct:
        text = text.replace(p, punct[p])

    # 한자어 변환
    text = hanja.translate(text, 'substitution')

    # 기사와 무관한 내용 제거
    text = text.rsplit('기자', 1)[0]
    text = text.rsplit('@', 1)[0]
    text = text.rsplit('.', 1)[0]+'.'
    text = text.split('▶')[0]
    text = text.split('☞')[0]
    text = text.split('[MBN')[0]
    text = text.split('[')[0]

    return text

def df2pre(text):
    # 불필요한 문자 제거 및 대체
    punct = {'…':'...',
            '“':'"', '”':'"',
            "‘":"'", "’":"'",
            "‘":"'", "’":"'",
            '\t':'', '\n':'', '\r':' ',
            '/':'',
            '▲':'', '■':'', '●':'', '○':'',
            '(사진)':''}
    for p in punct:
        text = text.replace(p, punct[p])


    # 한자어 변환
    text = hanja.translate(text, 'substitution')

    # 기사와 무관한 내용 제거
    text = text.rsplit('기자', 1)[0]
    text = text.rsplit('.', 1)[0]+'.'

    return text

def df3pre(text):
    # 불필요한 문자 제거 및 대체
    punct = {'…':'...',
           '“':'"', '”':'"',
           "‘":"'", "’":"'",
           "‘":"'", "’":"'",
           '\t':'', '\n':'', '\r':' ',
           '/':'',
           '▲':'', '■':'', '●':'', '○':'',
           '(사진)':'', '동영상 뉴스':''}
    for p in punct:
        text = text.replace(p, punct[p])

    # 한자어 변환
    text = hanja.translate(text, 'substitution')

    # 기사와 무관한 내용 제거
    text = text.rsplit('(영상', 1)[0]
    text = text.rsplit('@', 1)[0]
    text = text.rsplit('.', 1)[0]+'.'
    text = text.split('▶')[0]
    text = text.split('ⓒ')[0]
    text = text.rsplit('꿈을 담는 캔버스 채널A', 1)[0]

    return text


In [4]:
# print 해볼 때 전체 다 띄워보려고 쓰는 옵션
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)
pd.set_option('display.max_colwidth', None)

with open("data/naverNewsData.pickle",'rb') as f:
    raw = pickle.load(f)

raw['date'] = pd.to_datetime(raw['date'])

# 과거 데이터의 경우 양식이 굉장히 다름
data = raw[raw['date']>='2017-01-01'].copy()

df1 = data[(data['pressName']!='MBC') &
            (data['pressName']!='채널A') &
            (data['pressName']!='JTBC')&
            (data['pressName']!='연합뉴스TV')&
            (data['pressName']!='YTN')].copy()

df1.reset_index(drop=True, inplace=True)
df1['revisedArticle'] = 0

for i in range(len(df1)):
    text = df1['article'][i]
    df1['revisedArticle'][i] = df1pre(text)

#### df2
df2=data[(data['pressName']=='MBC')].copy()
df2.reset_index(drop=True, inplace=True)
df2['revisedArticle']=0

for i in range(len(df2)):
    text = df2['article'][i]
    df2['revisedArticle'][i] = df2pre(text)

#### df3
df3 = data[(data['pressName']=='채널A')|
            (data['pressName']=='JTBC')|
            (data['pressName']=='연합뉴스TV')|
            (data['pressName']=='YTN')].copy()
df3.reset_index(drop=True, inplace=True)
df3['revisedArticle']=0

for i in range(len(df3)):
    text = df3['article'][i]
    df3['revisedArticle'][i] = df3pre(text)

A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [5]:
revised_v1 = pd.concat([df1.drop(columns=['article']), df2.drop(columns=['article'])])
revised_v2 = pd.concat([revised_v1, df3.drop(columns=['article'])])

To remove this warning, run the following python code before doing dataframe operations:

    import ray
    ray.init()



In [6]:
revised_v2.head()

,pressName,ceoName,newsTitle,url,date,revisedArticle
0,조선일보,서경배,"올해 기업 총수 주식 성적표를 공개합니다, 1등은?",https://news.naver.com/main/read.nhn?mode=LSD&mid=sec&sid1=101&oid=023&aid=0003581759,2020-12-08,"국내 50대 그룹 총수 주식재산 증감액 한국CXO연구소국내 50대 그룹 총수 중 올 1월 초 대비 12월 초 기준 주식재산 상승률이 가장 컸던 주인공은 카카오 김범수 이사회 의장인 것으로 파악됐다. 기업분석 전문 한국CXO연구소는 8일 '국내 50대 그룹 총수의 2020년 1월 초 대비 12월 초 주식재산 증감 현황 분석' 결과를 발표했다. 조사 대상은 공정거래위원회가 자산 5조 원 이상 대기업 집단(그룹)으로 지정한 64곳 중 동일인(총수)이 있는 50대 그룹이다. 조사 대상에는 고(고) 삼성전자 이건희 전 회장과 현대차 정의선 회장 등 2명을 포함해 총 52명이다.이번 조사는 총수가 직접 주식을 보유한 상장사 보통주 주식으로 제한했고, 우선주는 조사에서 제외했다. 주식평가액은 올 1월 2일과 12월 2일 종가(종가)로 계산해 산출했다. 조사 결과에 의하면 52명의 그룹 총수 중 39명이 상장사 주식을 보유하고 있는 것으로 파악됐다. 39명의 총수의 올 1월 2일 기준 전체 주식평가액은 57조 6150억 원이었는데, 12월 2일에는 67조 1913억 원으로 9조 5695억 원(16.6%) 증가했다. 그룹 총수 20명은 1년 새 주식재산을 불렸다.주식평가액 증감 금액으로 살펴봤을 때 연초 대비 12월 초에 지분가치가 가장 크게 증가한 주인공은 삼성전자 이건희 전 회장이었다. 이 전 회장은 올 초만 해도 주식재산이 17조 3800억 원이었는데 12월 초에는 21조 397억 원으로, 3조 6597억 원(21.1%) 높아졌다.김범수 카카오 의장카카오카카오 김범수 의장도 1조 9068억 원에서 4조 6627억 원으로 2조 7560억 원이나 주식평가액이 상승했다. 1월 초 대비 12월 초 주식평가액 상승률로만 놓고 보면 김범수 의장이 144.5%로 50대 그룹 총수 중 1위다.현대차 정의선 회장도 올해 들어 주식재산이 1조 넘게 많아졌다. 2조 2268억 원에서 3조 2920억 원으로 1조 651억 원(47.8%) 증가한 것으로 집계됐다. 한국CXO연구소는 ""현대차 정 회장은 코로나19가 한창이던 상황에서 현대차와 현대모비스 주식을 꾸준히 매입해둔 것이 주식재산 증가에도 톡톡히 효과를 봤다""고 설명했다. 삼성전자 이재용 부회장은 9351억 원, 현대차 정몽구 명예회장은 8507억 원 늘었다.반면 아모레퍼시픽 서경배 회장은 코로나19 영향으로 주식재산이 1조 3000억 원 넘게 줄었다. 서 회장의 주식가치는 1월 초 4조 9976억 원에서 12월 초 3조 6352억 원으로 1조 3624억 원(-27.3%) 하락했다. SK 최태원 회장의 주식가치도 2716억 원(3조 3483억 원→3조 767억 원) 이상 감소했다. 신세계 이명희 회장과 한국테크놀로지그룹 조양래 회장은 올해 주식을 자녀에게 상속하다 보니 주식평가액 규모도 이전보다 40% 넘게 축소됐다.올 12월 초 기준으로 주식재산 1조 클럽에 가입한 50대 그룹 총수는 13명으로 조사됐다. 삼성전자 이건희 전 회장이 21조 원 이상으로 국내 주식부자 1위 자리를 지켰다. 삼성전자 이재용 부회장은 8조 2000억 원 이상으로 넘버2를 차지했다. 이어 3위 현대차 정몽구 명예회장, 4위 카카오 김범수 의장, 5위 아모레퍼시픽 서경배 회장 순으로 나타났다. 6~10위에는 각각 현대차 정의선 회장, SK 최태원 회장, 넷마블 방준혁 의장(2조 6015억 원), 셀트리온 서정진 회장, LG 구광모 회장(2조 180억 원)이 자리했다. 네이버 이해진 글로벌투자책임자(1조 7378억 원), 현대중공업 정몽준 아산재단 이사장(1조 1808억 원), CJ 이재현 회장(1조 695억 원)도 주식재산 1조 클럽에 가입했다.총수 보유 주식 상승 현황 한국CXO연구소이중 카카오 김범수 의장은 비상장 회사 지분 가치까지 합산할 경우 비공식적으로 주식부자 랭킹 2위까지 올라서는 것으로 파악됐다. 김범수 의장은 12월 초 기준 카카오 주식을 1250만 주 넘게 갖고 있는데 이 가치만 해도 4조 6000억 원 넘었다. 비상장사인 '케이큐브홀딩스'도 카카오 지분을 3조 7000억 원 상당 수준으로 보유 중이다. 케이큐브홀딩스는 김범수 의장이 지분을 100% 소유하고 있는 회사다. 케이큐브홀딩스가 보유한 카카오 주식가치까지 김 의장의 주식재산으로 합쳐 계산할 경우 8조 3701억 원으로 크게 높아진다. 이재용 부회장의 주식재산을 제치고 주식부자 넘버2 자리에 올라설 수 있는 수준이다.주가상승률 1,2위는 모두 두산 박정원 회장이 보유한 주식종목에서 나왔다. 두산퓨얼셀은 올 1월 2일만 해도 보통주 1주당 종가가 8800원이었는데 12월 2일에는 4만 6400원으로 427.3%나 고공 상승했다. 두산중공업도 162.7%(5730원→1만 5050원) 점프했다. 이어 SK디스커버리(152.7%), 셀트리온헬스케어(147.9%), 카카오(144.6%), 두산솔루스(139.7%), 효성중공업(133.1%), 코오롱글로벌(110.8%), 키다리스튜디오(103.7%) 주가가 1년 새 갑절 높아졌다. 이번 조사와 관련해 오일선 소장은 ""내년에는 그동안 이건희 회장이 지켜오던 주식부자 1위 자리를 이재용 부회장이 이어받을 것으로 예상됨에 따라 국내 주식부자 2~3위 자리를 놓고 그룹 총수 간 보이지 않는 치열한 경쟁이 예상된다""고 말했다."
1,조선일보,서경배,아모레퍼시픽G 대표에 김승환,https://news.naver.com/main/read.nhn?mode=LSD&mid=sec&sid1=101&oid=023&aid=0003575822,2020-11-13,㈜아모레퍼시픽그룹은 12일 김승환 인사조직실장(전무)을 대표이사 부사장에 선임했다. 김 부사장은 서경배 회장과 각자 대표이사를 맡는다.
2,조선일보,서경배,"코로나 직격탄 아모레, 51세 대표이사 파격 발탁",https://news.naver.com/main/read.nhn?mode=LSD&mid=sec&sid1=101&oid=023&aid=0003575696,2020-11-12,"아모레퍼시픽그룹 김승환 대표이사아모레퍼시픽그룹이 지주회사 대표이사에 50대 초반의 인사를 임명하고, 핵심 보직에 1970년생들을 대거 발탁하는 인사를 단행했다. 코로나로 영업이익이 반토막 나자, 세대 교체 카드를 꺼낸 것이다.아모레퍼시픽그룹은 12일 그룹 인사조직 실장 겸 아모레퍼시픽 인사조직 유닛장인 김승환(51) 전무를 아모레퍼시픽그룹 대표이사(부사장)에 내정했다고 밝혔다. 전임 배동현 대표(65)보다 14세 젊다. 서경배 회장과 각자 대표이사를 맡게 된 김 신임 대표는 2006년 아모레퍼시픽에 입사해 경영전략팀장, 아모레퍼시픽그룹 전략기획 디비전장, 그룹 인사조직 실장등을 거쳤다. 아모레퍼시픽 기술연구원 R&D 유닛장은 박영호(52) 상무가 전무로 승진하며 임명됐다.화장품 부문 주요 브랜드 책임자에는 1970년대생이 대거 발탁됐다. ㈜아모레퍼시픽의 라네즈 브랜드 유닛장에는 정혜진(45) 전무, 설화수 브랜드 유닛장에는 임중식(49) 상무가 발탁됐다. 중국 사업을 책임지는 ㈜아모레퍼시픽 중국 RHQ 부GM실장응 황영민(47) 상무가 맡는다.아모레퍼시픽 관계자는 ""사업 경쟁력을 제고하고, 위기를 타개해 재도약의 발판을 마련하기 위해 세대교체를 단행했다""고 밝혔다.아모레퍼시픽그룹은 코로나 여파로 지난 3분기 매출은 1조 2086억원, 영업이익은 610억원을 기록했다. 지난해 같은 기간 대비 매출은 23%, 영업이익은 49% 감소했다."
3,조선일보,서경배,"옵티머스 사태 연루된 靑 前행정관, 국감 불출석 사유서 늑장 공유한 與",https://news.naver.com/main/read.nhn?mode=LSD&mid=sec&sid1=100&oid=023&aid=0003570435,2020-10-22,"야 항의에 하루 뒤 팩스로 보내... 전행정관 ""

In [7]:
import pandas as pd
import numpy as np
from typing import List
from textrankr import TextRank
from konlpy.tag import Okt

class OktTokenizer:
    okt: Okt = Okt()

    def __call__(self, text: str) -> List[str]:
        tokens: List[str] = self.okt.pos(text, norm=True, stem=True, join=True)
        return tokens

In [8]:
content_list = revised_v2.revisedArticle

mytokenizer: OktTokenizer = OktTokenizer()
textrank: TextRank = TextRank(mytokenizer)

k: int = 3  # num sentences in the resulting summary
result_list = []

In [11]:
revised_v2.revisedArticle[0]

0    국내 50대 그룹 총수 주식재산 증감액 한국CXO연구소국내 50대 그룹 총수 중 올 1월 초 대비 12월 초 기준 주식재산 상승률이 가장 컸던 주인공은 카카오 김범수 이사회 의장인 것으로 파악됐다. 기업분석 전문 한국CXO연구소는 8일 '국내 50대 그룹 총수의 2020년 1월 초 대비 12월 초 주식재산 증감 현황 분석' 결과를 발표했다. 조사 대상은 공정거래위원회가 자산 5조 원 이상 대기업 집단(그룹)으로 지정한 64곳 중 동일인(총수)이 있는 50대 그룹이다. 조사 대상에는 고(고) 삼성전자 이건희 전 회장과 현대차 정의선 회장 등 2명을 포함해 총 52명이다.이번 조사는 총수가 직접 주식을 보유한 상장사 보통주 주식으로 제한했고, 우선주는 조사에서 제외했다. 주식평가액은 올 1월 2일과 12월 2일 종가(종가)로 계산해 산출했다. 조사 결과에 의하면 52명의 그룹 총수 중 39명이 상장사 주식을 보유하고 있는 것으로 파악됐다. 39명의 총수의 올 1월 2일 기준 전체 주식평가액은 57조 6150억 원이었는데, 12월 2일에는 67조 1913억 원으로 9조 5695억 원(16.6%) 증가했다. 그룹 총수 20명은 1년 새 주식재산을 불렸다.주식평가액 증감 금액으로 살펴봤을 때 연초 대비 12월 초에 지분가치가 가장 크게 증가한 주인공은 삼성전자 이건희 전 회장이었다. 이 전 회장은 올 초만 해도 주식재산이 17조 3800억 원이었는데 12월 초에는 21조 397억 원으로, 3조 6597억 원(21.1%) 높아졌다.김범수 카카오 의장카카오카카오 김범수 의장도 1조 9068억 원에서 4조 6627억 원으로 2조 7560억 원이나 주식평가액이 상승했다. 1월 초 대비 12월 초 주식평가액 상승률로만 놓고 보면 김범수 의장이 144.5%로 50대 그룹 총수 중 1위다.현대차 정의선 회장도 올해 들어 주식재산이 1조 넘게 많아졌다. 2조 2268억 원에서 3조 2920억 원으로 1조 651억 원(47.8%) 증가한 것으로 집계됐다. 한국CXO연구소는 "

In [12]:
summaries: List[str] = textrank.summarize(revised_v2.revisedArticle[0], k, verbose=True)

TypeError: expected string or bytes-like object

In [ ]:
for content in content_list:
    summaries: List[str] = textrank.summarize(content, k, verbose=True)
    result = '. '.join(summaries)
    result_list.append(result)

In [ ]:
# import pandas as pd
# import numpy as np
# from typing import List
# from textrankr import TextRank
# from konlpy.tag import Okt

# class OktTokenizer:
#     okt: Okt = Okt()

#     def __call__(self, text: str) -> List[str]:
#         tokens: List[str] = self.okt.pos(text, norm=True, stem=True, join=True)
#         return tokens

# def summarize(df):
#     content_list = df.revisedArticle

#     mytokenizer: OktTokenizer = OktTokenizer()
#     textrank: TextRank = TextRank(mytokenizer)

#     k: int = 3  # num sentences in the resulting summary
#     result_list = []

#     # if verbose = False, it returns a list
#     for content in content_list[:5]:
#         summaries: List[str] = textrank.summarize(content, k, verbose=False)
#         result = '. '.join(summaries)
#         result_list.append(result)

#     # print(result_list)
#     df['revisedArticle'] = result_list
#     return df

In [13]:
# revised = summarize(revised_v2.head())
revised_v2.to_csv('data/local_data_preprocessed.csv', encoding='UTF-8-sig')